In [1]:
from english_words import english_words_set
import pandas as pd
import numpy as np

In [2]:
# make a list of 5 lenght words without non-alphas, and no proper nouns
words5 = []

for word in english_words_set:
    if len(word) == 5 and word[0].islower() and word.isalpha():
        words5.append(word)

In [3]:
# this should be an empty list, checking to be sure we do not have non alpha chars in the words
print([i for i in words5 if not i.isalpha()])

[]


In [4]:
df_words = pd.DataFrame(words5)
df_words.columns=['words']

In [5]:
df_words['first'] = df_words['words'].str.slice(0,1)
df_words['second'] = df_words['words'].str.slice(1,2)
df_words['third'] = df_words['words'].str.slice(2,3)
df_words['fourth'] = df_words['words'].str.slice(3,4)
df_words['fifth'] = df_words['words'].str.slice(4,5)

In [6]:
df_words.head()

,words,first,second,third,fourth,fifth
0,torch,t,o,r,c,h
1,gauzy,g,a,u,z,y
2,shako,s,h,a,k,o
3,clock,c,l,o,c,k
4,lapel,l,a,p,e,l


In [7]:
aplhas = ['a','b','c','d','e','f','g','h','i','j','k','l','m','n','o','p','q','r','s','t','u','v','w','x','y','z']

In [8]:
dfalpha = pd.DataFrame(aplhas)
dfalpha.columns = ['letters']
dfalpha.set_index('letters', inplace=True)

In [9]:
# build the frequerncy table from each smaller dataframe
df_counts = df_words['first'].value_counts()
df_counts_second = df_words['second'].value_counts()
df_counts_third = df_words['third'].value_counts()
df_counts_fourth = df_words['fourth'].value_counts()
df_counts_fifth = df_words['fifth'].value_counts()

dfranks = pd.concat([dfalpha, df_counts], axis=1)
dfranks = pd.concat([dfranks, df_counts_second], axis=1)
dfranks = pd.concat([dfranks, df_counts_third], axis=1)
dfranks = pd.concat([dfranks, df_counts_fourth], axis=1)
dfranks = pd.concat([dfranks, df_counts_fifth], axis=1)

In [10]:
dfranks.head()

,first,second,third,fourth,fifth
a,157,334,334,199.0,93.0
b,203,18,56,25.0,10.0
c,224,42,61,162.0,41.0
d,107,19,78,74.0,110.0
e,72,262,197,261.0,468.0


In [11]:
dfranks['first']['a']

157

In [12]:
def freq_ranking(word):
    val = dfranks['first'][word[0]] + dfranks['second'][word[1]] +dfranks['third'][word[2]] + dfranks['fourth'][word[3]] +dfranks['fifth'][word[4]]
    return val

In [13]:
df_word_ranks = pd.DataFrame({'word' : [],'value':[]})

In [14]:
word_freq = {'word':'value'}
for word in words5:
    df_word_ranks.loc[len(df_word_ranks.index), 'word'] = word
    df_word_ranks.loc[len(df_word_ranks.index)-1, 'value'] = freq_ranking(word)

In [15]:
df_word_ranks.head()

,word,value
0,torch,968.0
1,gauzy,1034.0
2,shako,1026.0
3,clock,968.0
4,lapel,929.0


In [16]:
df_word_ranks.to_csv('word_ranks')

In [17]:
print (df_word_ranks['word'][df_word_ranks['value'].idxmax()])
print (df_word_ranks['value'][df_word_ranks['value'].idxmax()])

slate
1569.0


In [18]:
# here we determine the next best word based on results of the best word per wordle feedback
# how to determine which items were elimiated
#  ~ before a contains statement makes it a "not" I was lucky to find this
display(df_word_ranks.loc[(~df_word_ranks['word'].str.contains('t'))])

,word,value
1,gauzy,1034.0
2,shako,1026.0
3,clock,968.0
4,lapel,929.0
9,halve,1065.0
...,...,...
2465,quick,776.0
2466,chomp,780.0
2467,passe,1223.0
2468,alley,1139.0


In [19]:
grey = ['s','l','a','t','e']

In [20]:
df_sub = df_word_ranks
for l in grey:
    print(l)
    print(str(len(df_sub.index)))
    df_sub = (df_sub.loc[(~df_sub['word'].str.contains(l))])
df_sub

s
2470
l
1780
a
1240
t
738
e
495


,word,value
16,hough,803.0
31,cocky,1030.0
42,chock,919.0
57,fuzzy,752.0
59,wring,832.0
...,...,...
2439,ivory,858.0
2446,groin,974.0
2460,bunch,881.0
2465,quick,776.0


In [21]:
print (df_sub['word'][df_sub['value'].idxmax()])
print (df_sub['value'][df_sub['value'].idxmax()])

crony
1308.0


In [23]:
def remove_notcontains(grey,df):
    df_sub = df
    for l in grey:
        df_sub = (df_sub.loc[(~df_sub['word'].str.contains(l))])
    return df_sub

In [24]:
def remove_by_contains(letters,df):
    df_sub = df
    for l in letters:
        df_sub = (df_sub.loc[(df_sub['word'].str.contains(l))])
    return df_sub

In [25]:
def remove_by_location(loc,letter,df):
    df_sub = df
    for l in grey:
        df_sub = (df_sub.loc[(df_sub['word'].str[loc]== letter)])
    return df_sub

In [26]:
grey = input()
df_options = df_word_ranks

In [27]:
df_options = remove_notcontains(grey,df_options)
print(df_options.head())
next_best = (df_options['word'][df_options['value'].idxmax()])
print(next_best)

     word   value
16  hough   803.0
31  cocky  1030.0
42  chock   919.0
57  fuzzy   752.0
59  wring   832.0
crony


In [28]:
confirmed = 'cr'

In [29]:
df_options = remove_by_contains(confirmed,df_options)
print(df_options.head())
next_best = (df_options['word'][df_options['value'].idxmax()])
print(next_best)

      word   value
141  chord   935.0
379  choir   959.0
399  crook   999.0
410  crump   811.0
414  brick  1012.0
crony


In [30]:
loc_letters = input('enter letter and locations in pairs [r3t4]')
print(loc_letters)

r1


In [31]:
for i in range(0,int(len(loc_letters)),2):
    print(str(i))
    print(loc_letters[i])
    print(loc_letters[i+1])
    df_options = remove_by_location(loc=int(loc_letters[i+1]),letter=str(loc_letters[i]),df=df_options)
    print(df_options.head())
    next_best = (df_options['word'][df_options['value'].idxmax()])
    print(next_best)

0
r
1
      word   value
399  crook   999.0
410  crump   811.0
414  brick  1012.0
569  crock  1024.0
635  crimp   894.0
crony


In [46]:
# make the testing loop
# starting DF df_word_ranks

round = 0
df_options = df_word_ranks

while round <= 6:
    round += 1 
    if round == 1:
        print('initial guess')
        word = input('enter your guess:')
        pass

    # try:
    if round >= 2:
        print('give the results of first guess')
        not_char = input('enter eliminated letters:[l,e,t,t,e,r]')
        is_char = input('enter confirmed letters:[y,e,s]')
        loc_letters = input('enter confirmed locations: [letter,location]')

        # eliminate options, and show the reduced best 5 choices
        if len(not_char)>1:
            df_options = remove_notcontains(not_char,df_options)
            # print(df_options.head())
        if len(is_char)>0:
            df_options = remove_by_contains(is_char,df_options)
            # print(df_options.head())
            if len(loc_letters)>0:
                # remove by letters in correct location
                for i in range(0,int(len(loc_letters)),2):
                    df_options = remove_by_location(loc=int(loc_letters[i+1]),letter=str(loc_letters[i]),df=df_options)
                # print(df_options.head())

        next_best = (df_options['word'][df_options['value'].idxmax()])
    print(df_options.head())
    print('from the results of the last guess')
    print('next best option is ' + str(next_best))
    print(str(round))
        
    # except:
    #     print(word)

initial guess
    word   value
0  torch   968.0
1  gauzy  1034.0
2  shako  1026.0
3  clock   968.0
4  lapel   929.0
from the results of the last guess
next best option is brick
1
give the results of first guess


KeyboardInterrupt: Interrupted by user